In [33]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [34]:
ostro_dir = 'Osteosarcoma-UT'



# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [35]:
from sklearn.model_selection import train_test_split
import os

# Define the root directory containing subfolders for each class
root_dir = 'Osteosarcoma-UT'

# Get the list of class names
classes = os.listdir(root_dir)
print(classes)

# Split the data into training and validation sets
train_data = []
val_data = []
for cls in classes:
    cls_dir = os.path.join(root_dir, cls)
    images = [os.path.join(cls_dir, img) for img in os.listdir(cls_dir)]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    train_data.extend([(img, cls) for img in train_images])
    val_data.extend([(img, cls) for img in val_images])

# Further split the training data into training and testing sets
train_images, test_images = train_test_split(train_data, test_size=0.2, random_state=42)

# Now you have train_images, val_data, and test_images containing paths to images for each set



['Viable', 'Non-Tumor', 'Non-Viable-Tumor']


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame with two columns: 'file_path' and 'label'
train_df = pd.DataFrame(train_images, columns=['file_path', 'label'])
val_df = pd.DataFrame(val_data, columns=['file_path', 'label'])
test_df = pd.DataFrame(test_images, columns=['file_path', 'label'])

# Split the data into file paths and labels
train_data = train_df['file_path']
train_labels = train_df['label']

val_data = val_df['file_path']
val_labels = val_df['label']

test_data = test_df['file_path']
test_labels = test_df['label']

# You can create separate data generators for each set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 731 validated image filenames belonging to 3 classes.
Found 230 validated image filenames belonging to 3 classes.
Found 183 validated image filenames belonging to 3 classes.


In [37]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD


img_width, img_height = 224, 224
batch_size = 32

# Load pre-trained VGG19 without top layers
base_model = VGG19(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # You can add more dense layers if needed
predictions = Dense(3, activation='softmax')(x) 


model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=SGD(lr=0.001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

In [39]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=[early_stopping])

Epoch 1/100


2024-02-22 12:39:55.217550: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


23/23 [==============================] - 20s 805ms/step - loss: 1.0953 - accuracy: 0.4542 - val_loss: 0.8824 - val_accuracy: 0.5348
Epoch 2/100
23/23 [==============================] - 19s 801ms/step - loss: 0.7351 - accuracy: 0.6908 - val_loss: 0.5914 - val_accuracy: 0.7913
Epoch 3/100
23/23 [==============================] - 18s 782ms/step - loss: 0.6731 - accuracy: 0.7237 - val_loss: 0.8239 - val_accuracy: 0.7217
Epoch 4/100
23/23 [==============================] - 18s 790ms/step - loss: 0.6498 - accuracy: 0.7620 - val_loss: 0.5003 - val_accuracy: 0.8261
Epoch 5/100
23/23 [==============================] - 19s 813ms/step - loss: 0.6310 - accuracy: 0.7524 - val_loss: 0.5639 - val_accuracy: 0.7565
Epoch 6/100
23/23 [==============================] - 20s 864ms/step - loss: 0.5746 - accuracy: 0.7674 - val_loss: 0.4695 - val_accuracy: 0.8261
Epoch 7/100
23/23 [==============================] - 21s 916ms/step - loss: 0.5214 - accuracy: 0.8016 - val_loss: 0.5477 - val_accuracy: 0.7783
Epoc

In [40]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

5/5 [==============================] - 4s 645ms/step - loss: 0.3376 - accuracy: 0.9062
Test accuracy: 0.90625


In [41]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score

# Assuming you already have your test generator and model trained

# Evaluate the model using test generator
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

# Get predictions and true labels
# Ensure that steps is set to get the correct number of samples
y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)
y_true = test_generator.classes

# Only take the necessary number of samples from y_true
y_true = y_true[:len(y_pred)]

# Calculate macro-averaged F1-score
macro_f1 = f1_score(y_true, y_pred.argmax(axis=1), average='macro')
print("Macro-averaged F1-score:", macro_f1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred.argmax(axis=1))
print("Confusion matrix:\n", cm)

# Calculate AUC-ROC score for each class
y_onehot = np.zeros((len(y_true), len(np.unique(y_true))))
y_onehot[np.arange(len(y_true)), y_true] = 1
auc_roc_scores = roc_auc_score(y_onehot, y_pred, multi_class='ovr')
print("AUC-ROC scores for each class:", auc_roc_scores)

# Calculate average AUC-ROC score
average_auc_roc = np.mean(auc_roc_scores)
print("Average AUC-ROC score:", average_auc_roc)


5/5 [==============================] - 3s 636ms/step - loss: 0.3376 - accuracy: 0.9062
Test accuracy: 0.90625


/var/folders/yd/6l3qbbqj2qx_y_lxrr45xd480000gn/T/ipykernel_24207/3608557048.py:12: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)


Macro-averaged F1-score: 0.8979066022544284
Confusion matrix:
 [[86  7  0]
 [ 2 39  3]
 [ 3  2 41]]
AUC-ROC scores for each class: 0.9730004010403084
Average AUC-ROC score: 0.9730004010403084
